In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import cv2
import time
from skimage import morphology
import seaborn as sns
import collections
import random
plt.rcParams['figure.figsize'] = (20,12)
%config InlineBackend.figure_format = 'retina'


In [72]:
def randomize_in_ROI(target_images_list, ROI_image, randomize_size=1, peripheral_shuffle=True):
    """
        Args(引数)
            target_images_list: list型, 各画像は2次元配列
            ROI_image: numpy.ndarray型, 2次元配列でbinarize画像
            randomize_size: int型, シャッフルしたいサイズの正方形の一辺の長さ
            peripheral_shuffle: bool型, randomize_sizeでシャッフルした後、サイズ的にシャッフルできなかった箇所を1pixel単位でシャッフルするかどうか
        
        Return(返り値)
            randomized_imgs_list: list型, target_imagesで入っていた画像の順でrandomizeした画像を返す
    """
    
    H = ROI_image.shape[0] // randomize_size
    W = ROI_image.shape[1] // randomize_size
    ROI_image_copy = (ROI_image>0).astype(int)
    
    # ROI_imageをrandomize_sizeを一辺とする正方形で区切り、各正方形においてROIに含まれている正方形のみを抽出する
    grid_list = list()
    for h in range(H):
        for w in range(W):
            if np.min(ROI_image_copy[h*randomize_size:(h+1)*randomize_size, w*randomize_size:(w+1)*randomize_size]) == 1:
                grid_list.append([h, w])
                
    shuffle_grid_list = random.sample(grid_list, len(grid_list))
    
    if peripheral_shuffle:
        # randomize_sizeを一辺とする正方形でのshuffleに含まれていないROIのpixelを抽出する
        for h, w in grid_list:
            ROI_image_copy[h*randomize_size:(h+1)*randomize_size, w*randomize_size:(w+1)*randomize_size] -= 1

        ROI_image_coords = ROI_image_copy.nonzero()
        pixel_list = list( [h, w] for h, w in zip(ROI_image_coords[0], ROI_image_coords[1]) )
        shuffle_pixel_list = random.sample(pixel_list, len(pixel_list))

    
    # shuffleを実行する
    randomized_imgs_list = list()
    for i, target_image in enumerate(target_images_list):
        print(f"{str(i+1)}枚目のrandom shuffleを開始します")
        randomized_img = np.zeros_like(target_image)
        for grid_i, shuffle_grid_i in zip(grid_list, shuffle_grid_list):
            randomized_img[
                shuffle_grid_i[0]*randomize_size:(shuffle_grid_i[0]+1)*randomize_size, 
                shuffle_grid_i[1]*randomize_size:(shuffle_grid_i[1]+1)*randomize_size
            ] = target_image[
                grid_i[0]*randomize_size:(grid_i[0]+1)*randomize_size, 
                grid_i[1]*randomize_size:(grid_i[1]+1)*randomize_size
            ]
        if peripheral_shuffle:
            for pixel_i, shuffle_pixel_i in zip(pixel_list, shuffle_pixel_list):
                randomized_img[shuffle_pixel_i[0], shuffle_pixel_i[1]] = target_image[pixel_i[0], pixel_i[1]]
        print(f"{str(i+1)}枚目のrandom shuffleが終わりました")
        randomized_imgs_list.append(randomized_img)

        
    return randomized_imgs_list

In [113]:
threshold_O = 120
threshold_F = 400 

for i in range(1,4):
    file_name = r"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20  - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_"
    Mito = cv2.imread(file_name + str(i) + ".tif_OxPHOS.tif",cv2.IMREAD_ANYDEPTH)
    FKBP8 = cv2.imread(file_name + str(i) + ".tif_FKBP8.tif",cv2.IMREAD_ANYDEPTH) 
   
    ret_M ,th_M = cv2.threshold(Mito,threshold_O,65535,cv2.THRESH_BINARY)
    th_M = th_M/65535

    ret_F ,th_F = cv2.threshold(FKBP8,threshold_F,65535,cv2.THRESH_BINARY)
    th_F = th_F/65535
    FKBP8 = FKBP8 - threshold_F
    FKBP8 = FKBP8 * th_F
    
    # Converting to 8bit image
    th_F = (th_F).astype('uint8')
    th_M = (th_M).astype('uint8')

    #Outside of Mito
    th_M_conv = np.zeros_like(th_M)
    th_M_conv = np.where(th_M == 1, 0, 1)
    th_M_conv = (th_M_conv).astype('uint8')
    
    # Applying cv2.connectedComponents()
    retval_F, labels_F, stats_F, centroids_F = cv2.connectedComponentsWithStats(th_F)
        
    centroids_F_img = np.zeros_like(th_F)
    for coordinate in centroids_F[1:]:
        center = (int(coordinate[0]), int(coordinate[1]))
        centroids_F_img = cv2.circle(centroids_F_img, center, 0, (255, 255, 255)) 
    
    centroids_F_on_M_img = centroids_F_img * th_M
    centroids_F_out_M_img = centroids_F_img * th_M_conv
    cv2.imwrite(file_name + str(i) + "_FKBP8_on_mito_centroids.tif", centroids_F_on_M_img)
    cv2.imwrite(file_name + str(i) + "_FKBP8_off_mito_centroids.tif", centroids_F_out_M_img)
    
    randomized_imgs_list = randomize_in_ROI(
    target_images_list=[centroids_F_on_M_img], 
    ROI_image=th_M, 
    randomize_size=1, 
    peripheral_shuffle=True) 
    centroids_F_on_M_shuffled_img = randomized_imgs_list[0]
    cv2.imwrite(file_name + str(i) + "_FKBP8_on_mito_centroids_randomized.tif", centroids_F_on_M_shuffled_img)
    
    randomized_imgs_list = randomize_in_ROI(
    target_images_list=[centroids_F_out_M_img], 
    ROI_image=th_M_conv, 
    randomize_size=1, 
    peripheral_shuffle=True)
    centroids_F_out_M_shuffled_img = randomized_imgs_list[0]
    cv2.imwrite(file_name + str(i) + "_FKBP8_off_mito_centroids_randomized.tif", centroids_F_out_M_shuffled_img)

1枚目のrandom shuffleを開始します
1枚目のrandom shuffleが終わりました
1枚目のrandom shuffleを開始します
1枚目のrandom shuffleが終わりました
1枚目のrandom shuffleを開始します
1枚目のrandom shuffleが終わりました
1枚目のrandom shuffleを開始します
1枚目のrandom shuffleが終わりました
1枚目のrandom shuffleを開始します
1枚目のrandom shuffleが終わりました
1枚目のrandom shuffleを開始します
1枚目のrandom shuffleが終わりました


In [2]:
def main1(file_name, threshold_P, threshold_O, threshold_F):
    obj_on_mito = []
    obj_out_mito = []
    obj_r_on_mito = []
    obj_r_out_mito = []
    obj_on_mito_list = []
    obj_out_mito_list = []
    obj_r_on_mito_list = []
    obj_r_out_mito_list = [] 

    for i in range(1,4):
        PDZD8 =  cv2.imread(file_name + str(i) + ".tif_PDZD8.tif",cv2.IMREAD_ANYDEPTH)
        Mito = cv2.imread(file_name + str(i) + ".tif_OxPHOS.tif",cv2.IMREAD_ANYDEPTH)
        FKBP8 = cv2.imread(file_name + str(i) + ".tif_FKBP8.tif",cv2.IMREAD_ANYDEPTH) 
        centroids_F_on_M_img = cv2.imread(file_name + str(i) + "_FKBP8_on_mito_centroids.tif", 0)
        centroids_F_out_M_img = cv2.imread(file_name + str(i) + "_FKBP8_off_mito_centroids.tif", 0)
        centroids_F_on_M_randomized_img = cv2.imread(file_name + str(i) + "_FKBP8_on_mito_centroids_randomized.tif", 0)
        centroids_F_out_M_randomized_img = cv2.imread(file_name + str(i) + "_FKBP8_off_mito_centroids_randomized.tif", 0)

        ret_M ,th_M = cv2.threshold(Mito,threshold_O,65535,cv2.THRESH_BINARY)
        th_M = th_M/65535

        ret_F ,th_F = cv2.threshold(FKBP8,threshold_F,65535,cv2.THRESH_BINARY)
        th_F = th_F/65535
        FKBP8 = FKBP8 - threshold_F
        FKBP8 = FKBP8 * th_F
    
        ret_P ,th_P = cv2.threshold(PDZD8,threshold_P,65535,cv2.THRESH_BINARY)
        th_P = th_P/65535
        PDZD8 = PDZD8 - threshold_P
        PDZD8 = PDZD8 * th_P
    
        # Converting to 8bit image
        th_P = (th_P).astype('uint8')
        th_F = (th_F).astype('uint8')
        th_M = (th_M).astype('uint8')

        #Outside of Mito
        th_M_conv = np.zeros_like(th_M)
        th_M_conv = np.where(th_M == 1, 0, 1)
        th_M_conv = (th_M_conv).astype('uint8')

        # PDZD8 on Mito
        th_P_on_M = th_P * th_M
        #PDZD8 outside of Mito
        th_P_out_M = th_P * th_M_conv
        # FKBP8 on Mito
        th_F_on_M = th_F * th_M
        # FKBP8 outside of Mito
        th_F_out_M = th_F * th_M_conv
    
        # Applying cv2.connectedComponents()
        retval_P, labels_P, stats_P, centroids_P = cv2.connectedComponentsWithStats(th_P)
    
        centroids_P_img = np.zeros_like(th_F)
        for coordinate in centroids_P[1:]:
            center = (int(coordinate[0]), int(coordinate[1]))
            centroids_P_img = cv2.circle(centroids_P_img, center, 0, (255, 255, 255))    

        centroids_P_on_M_img = centroids_P_img * th_M
        centroids_P_out_M_img = centroids_P_img * th_M_conv 
    
        centroids_F_on_M_conv = np.zeros_like(th_F)
        centroids_F_on_M_conv = np.where(centroids_F_on_M_img == 255, 0, 1)
        centroids_F_on_M_conv = (centroids_F_on_M_conv).astype('uint8')
        F_on_mito_dist = cv2.distanceTransform(centroids_F_on_M_conv, cv2.DIST_L2, 5)
    
        centroids_F_out_M_conv = np.zeros_like(th_F)
        centroids_F_out_M_conv = np.where(centroids_F_out_M_img == 255, 0, 1)
        centroids_F_out_M_conv = (centroids_F_out_M_conv).astype('uint8')
        F_out_mito_dist = cv2.distanceTransform(centroids_F_out_M_conv, cv2.DIST_L2, 5)
        
        centroids_F_on_M_randomized_conv = np.zeros_like(th_F)
        centroids_F_on_M_randomized_conv = np.where(centroids_F_on_M_randomized_img == 255, 0, 1)
        centroids_F_on_M_randomized_conv = (centroids_F_on_M_randomized_conv).astype('uint8')
        F_on_mito_r_dist = cv2.distanceTransform(centroids_F_on_M_randomized_conv, cv2.DIST_L2, 5)
    
        centroids_F_out_M_randomized_conv = np.zeros_like(th_F)
        centroids_F_out_M_randomized_conv = np.where(centroids_F_out_M_randomized_img == 255, 0, 1)
        centroids_F_out_M_randomized_conv = (centroids_F_out_M_randomized_conv).astype('uint8')
        F_out_mito_r_dist = cv2.distanceTransform(centroids_F_out_M_randomized_conv, cv2.DIST_L2, 5)

        # Distance from PDZD8 to nearest FKBP8 on Mito
        obj_on_mito = np.where(centroids_P_on_M_img == 255, F_on_mito_dist, 10000)
        obj_on_mito = np.ravel(obj_on_mito)
        obj_on_mito = np.delete(obj_on_mito, np.where(obj_on_mito == 10000)[0])
    
        # Distance from PDZD8 to nearest FKBP8 off Mito
        obj_out_mito = np.where(centroids_P_out_M_img == 255, F_out_mito_dist, 10000)
        obj_out_mito = np.ravel(obj_out_mito)
        obj_out_mito = np.delete(obj_out_mito, np.where(obj_out_mito == 10000)[0])
        
        # Distance from PDZD8 to nearest r_FKBP8 on Mito
        obj_r_on_mito = np.where(centroids_P_on_M_img == 255, F_on_mito_r_dist, 10000)
        obj_r_on_mito = np.ravel(obj_r_on_mito)
        obj_r_on_mito = np.delete(obj_r_on_mito, np.where(obj_r_on_mito == 10000)[0])
    
        # Distance from PDZD8 to nearest r_FKBP8 off Mito
        obj_r_out_mito = np.where(centroids_P_out_M_img == 255, F_out_mito_r_dist, 10000)
        obj_r_out_mito = np.ravel(obj_r_out_mito)
        obj_r_out_mito = np.delete(obj_r_out_mito, np.where(obj_r_out_mito == 10000)[0])
    
        obj_on_mito_list = np.append(obj_on_mito_list, obj_on_mito)
        obj_out_mito_list = np.append(obj_out_mito_list, obj_out_mito)
        obj_r_on_mito_list = np.append(obj_r_on_mito_list, obj_r_on_mito)
        obj_r_out_mito_list = np.append(obj_r_out_mito_list, obj_r_out_mito)

    ans = [[obj_on_mito_list], [obj_out_mito_list], [obj_r_on_mito_list], [obj_r_out_mito_list]]
    
    return ans

In [3]:
df1_1 = pd.DataFrame(data=None, index=None, columns= ["obj_on_mito_list","obj_out_mito_list","obj_r_on_mito_list","obj_r_out_mito_list"], dtype=None, copy=False)
for i in [1,6,7,9,10]:
    file_name = r"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 " + str(i) + " - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_"
    threshold_P = 50
    threshold_O = 120
    threshold_F = 300 
    df1_1.loc[file_name] = main1(file_name, threshold_P, threshold_O, threshold_F)

In [4]:
df1_2 = pd.DataFrame(data=None, index=None, columns= ["obj_on_mito_list","obj_out_mito_list","obj_r_on_mito_list","obj_r_out_mito_list"], dtype=None, copy=False)
for i in [1,2,4,5]:
    file_path = r"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 " + str(i) + " - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_"
    threshold_P = 40
    threshold_O = 120
    threshold_F = 400
    df1_2.loc[file_path] = main1(file_path,threshold_P,threshold_O,threshold_F)

In [5]:
df1 = pd.concat([df1_1, df1_2])
df1

,obj_on_mito_list,obj_out_mito_list,obj_r_on_mito_list,obj_r_out_mito_list
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 1 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_","[[24.38134765625, 16.393798828125, 30.77209472...","[[58.519378662109375, 37.196746826171875, 28.1...","[[20.5906982421875, 14.193756103515625, 14.990...","[[36.393798828125, 9.393798828125, 23.39379882..."
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 6 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_","[[8.78759765625, 4.393798828125, 2.79998779296...","[[2.0, 21.387542724609375, 21.775177001953125,...","[[20.393798828125, 8.1968994140625, 15.9813842...","[[56.94720458984375, 23.390625, 42.947265625, ..."
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 7 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_","[[34.968994140625, 3.1968994140625, 12.5999450...","[[15.0, 22.39990234375, 56.193572998046875, 29...","[[41.56890869140625, 15.98138427734375, 3.5968...","[[19.1968994140625, 17.0, 31.181396484375, 43...."
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 9 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_","[[5.793792724609375, 11.5906982421875, 6.39688...","[[15.181396484375, 25.5875244140625, 30.584411...","[[10.5906982421875, 23.5751953125, 8.196899414...","[[20.978271484375, 22.78759765625, 14.59069824..."
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 10 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_","[[4.199981689453125, 2.1968994140625, 4.996887...","[[19.9844970703125, 10.5906982421875, 12.0, 17...","[[42.7720947265625, 12.1968994140625, 21.57519...","[[23.181396484375, 23.196807861328125, 30.7782..."
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 1 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_","[[35.78759765625, 6.393798828125, 11.393798828...","[[20.181365966796875, 12.384490966796875, 10.5...","[[48.968994140625, 18.199920654296875, 14.3937...","[[4.393798828125, 8.393798828125, 55.575195312..."
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 2 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_","[[20.7720947265625, 6.1968994140625, 8.3937988...","[[6.396881103515625, 16.9844970703125, 4.39379...","[[10.5906982421875, 12.79376220703125, 9.39379...","[[9.78759765625, 38.187469482421875, 29.378295..."
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 4 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_","[[4.99688720703125, 4.99688720703125, 7.393798...","[[63.990447998046875, 30.390594482421875, 19.1...","[[31.181396484375, 3.596893310546875, 18.37829...","[[16.58447265625, 38.747283935546875, 59.18737..."
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 5 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_","[[12.393798828125, 2.1968994140625, 21.3937988...","[[51.5751953125, 79.5751953125, 14.39379882812...","[[7.0, 3.1968994140625, 8.393798828125, 24.590...","[[9.1968994140625, 17.1968994140625, 23.799896..."


In [6]:
df1.to_csv("FKBP8_distance_list.csv")

In [160]:
x = np.array([24.38139534, 16.39379883, 30.77209473, 16.19689751, 15.59069824,
       19.19689941,  3.        , 10.79069901,  6.19689989, 18.57519722,
        9.19689941, 20.57519722, 63.19689941, 17.18759727, 15.        ,
       13.19689941, 10.39379883, 17.18139839,  9.39379978,  6.        ,
       12.39379883, 18.19999695,  1.        , 15.98449898, 18.98449707,
        7.99069977, 12.3844986 , 12.39379883, 24.38139534,  6.19689989,
        6.19689989, 19.18139648,  2.19689989,  4.19999981,  2.79999995,
       10.39379883,  7.1937995 , 25.3782959 , 13.39689827, 17.        ,
       23.57519531, 23.97209549,  5.39379978, 21.19689941,  3.19689989,
       11.99689865, 15.39379978, 14.38759804, 25.39379501,  8.19689941,
       18.18139839,  9.78759956, 17.78759766,  4.39379978, 10.98449898,
       10.        ,  8.39379978, 33.36279297, 18.78139687, 17.59689713,
       18.57519722,  9.59070015,  8.39999962, 18.37829781, 27.18139458,
       20.19689941, 11.5875988 , 19.57829666, 16.39069748, 15.19689941,
        9.59070015, 18.78139687, 24.3782959 , 15.19689941,  4.39379978,
       13.98449898,  2.19689989,  8.19689941, 15.37829781, 20.78759766,
        7.59069967,  7.39379978, 23.98449707, 33.57829285,  5.19689989,
       45.16588974, 14.19689941, 20.98449707, 16.18139839, 29.36279488,
       13.59069824,  5.5999999 , 20.59069633, 10.39379883, 17.7906971 ,
       17.7906971 , 26.18139648,  8.78759956, 17.19689941, 31.98449326,
       18.37829781,  4.        , 10.79069901,  5.39379978, 11.59070015,
       18.59069824, 18.59069824, 14.79689789, 17.37829781,  5.39379978,
       11.78759956, 13.18139839, 52.96899414,  9.39379978,  5.19689989,
        4.39379978, 15.98139763,  7.1937995 ,  5.19689989, 19.38449669,
       25.56589508, 15.59069824,8.78759956,  4.39379978,  2.79999995, 
       21.96899605, 11.39379883, 5.5999999 , 24.57519722, 22.18139648, 
       20.98449707, 24.79069519, 15.39379883, 11.39379883,  1.39999998,  
       9.19689941, 10.78759956, 8.        ,  6.59069967, 10.39379978, 
       10.98449898, 15.   , 8.59379959, 10.59070015, 18.39379883, 
       12.19069862,  9.19689941, 15.98139763, 18.        ,  9.79999924, 
       25.56589508, 26.39379883, 8.78759956, 33.77209473,  5.39379978,  
       6.3968997 ,  6.19689989, 10.19689941, 13.78449821,  8.19689941, 
       18.19689941,  2.79999995, 23.79999542, 16.19689941,  4.9968996 ,  
       3.59689999, 16.78759766, 10.79069901, 13.39379883,  4.19689989,  
       5.79379988, 15.98139763, 17.59069824,  8.78759956,  3.59689999,  
       9.39069939, 17.57519722, 4.39379978, 12.3844986 ,  8.59379959,  
       7.1937995 , 21.78759766, 8.59070015,  7.59069967,  3.19689989, 
       14.58139801, 11.5875988 , 15.59070015,  3.59689999, 19.18139648,  
       3.        ,  1.39999998, 8.59070015, 15.18449783, 14.18139839,  
       3.59689999,  5.19689989, 4.39379978,  9.39069939,  9.79999924, 
       15.19689941,  4.19689989, 4.9968996 ,  3.59689999, 18.98449707, 
       13.18139839,  3.        , 5.5999999 , 18.99689674, 11.19689941,  
       8.59379959, 15.59069824, 11.39379883, 10.59689903, 14.98449898, 
       16.        , 22.39379883, 22.18139648, 12.78759956, 31.96279335, 
       20.59069633, 34.96899414,  3.19689989, 12.59999847,  8.39379978,  
       8.59379959,
       14.        ,  9.19689941,  4.39379978,  4.19689989, 15.78759766,
        4.19999981, 10.79069901, 12.59999847,  4.39379978,  3.        ,
        3.19689989, 14.39379883,  7.59069967,  4.9968996 , 19.39379883,
        2.19689989, 19.98449707, 25.17829514,  9.39379978,  7.99069977,
       32.99689484,  2.19689989, 11.39379883,  8.39379978,  5.79379988,
        8.        ,  3.19689989,  7.1937995 ,  7.59069967,  4.9968996 ,
       13.39379883,  6.19689989, 13.39689827, 16.77829742, 14.18139839,
       16.58449745, 23.17519569, 19.19689941, 21.77519608, 22.3782959 ,
        9.79999924,  1.39999998,  2.19689989, 13.19689941, 11.39379883,
        6.3968997 , 24.96899605,  4.9968996 , 11.39379883, 38.95349121,
        7.99069977, 20.77209663, 10.18759918, 13.78449821,  8.78759956,
        9.79999924,  8.19689941, 21.96899605, 17.98449707, 52.55348969,
       44.54109192, 10.79069901,  4.19689989, 16.39069748,  8.39379978,
        7.1937995 , 11.98449898, 5.79379988, 11.59070015,  6.3968997 , 10.59070015, 16.78759956,
        3.19689989,  6.3968997 , 13.18139839,  2.19689989, 21.96899605,
        4.19689989,  9.59070015, 11.5875988 , 13.78759956, 10.78759956,
       22.57519722, 16.9937973 ,  1.        ,  8.59070015,  3.59689999,
       18.97519684,  4.19689989, 11.5875988 , 10.18759918, 11.98449898,
        6.        ,  7.59069967,  4.39379978, 12.39379883,  4.9968996 ,
       11.        , 16.37829781, 10.19689941, 12.19069862,  8.59070015,
        7.39379978,  2.        , 11.19689941, 14.98449898,  4.39379978,
       18.59070015, 10.        , 10.79069901,  6.39379978,  3.19689989,
        7.1937995 , 12.19689941, 13.98449898,  7.7968998 ,  5.19689989,
       17.59689713, 11.        ,  6.        , 12.        , 10.79069901,
       20.18139839, 22.77209663, 16.39069748, 10.79069901, 16.58449745,
       13.39689827, 21.38759613, 23.79999542,  4.19999981,  5.79379988,
       12.98449898,  9.78759956, 10.59689903, 15.39999771, 14.38759804,
        6.39379978, 17.59069824, 18.18139839, 14.79689789, 14.99069786,
       10.39379883, 11.99689865, 12.19069862,  6.        , 11.5875988 ,
        2.        , 10.19689941,  9.19689941,  7.59069967, 12.79379845,
        6.19689989,  9.99379921,  4.39379978,  4.39379978, 12.19069862,
       12.39379883, 16.59069824, 15.78759956, 20.18139648, 10.98449898,
       22.57209587,  8.59379959, 12.        ,  7.39379978,  4.19689989,
       10.98449898,  8.19689941,  6.3968997 ,  7.59069967,  1.        ,
       13.18139839,  2.19689989, 16.        , 11.99689865,  9.19689941,
       18.78139687, 17.39379883, 12.59070015, 4.19999981,  2.19689989,  4.9968996 , 13.78449821, 10.79069901,
        6.59069967,  8.59379959, 25.39379501, 15.18449783,  7.        ,
       11.5875988 ,  4.39379978,  4.        ,  3.        ,  9.19689941,
        8.78759956,  6.39379978,  8.19689941,  6.19689989,  6.39379978,
       16.9937973 , 19.19689941,  4.19689989, 10.59070015,  4.19689989,
        4.9968996 ,  4.19689989,  5.39379978, 11.39379883,  2.19689989,
        5.79379988,  7.        ,  2.79999995,  5.19689989,  2.        ,
        7.7968998 ,  4.9968996 ,  5.39379978,  9.39069939, 11.98449898,
        8.59379959,  6.59069967,  9.59070015, 10.        , 17.18139839,
        6.59069967, 20.18139648, 11.39379883,  5.79379988, 13.19689941,
       14.79689789, 11.59070015, 19.18139839, 19.38449669, 18.97519684,
        6.39379978, 14.78759766, 14.18139839, 10.18759918,  6.3968997 ,
        5.19689989,  4.        ,  7.19689989, 10.        ,  7.        ,
        7.19689989, 20.37829781, 10.39379883,  1.        ,  3.19689989,
        4.19999981, 11.99689865,  4.9968996 , 14.19689941, 13.18139839,
       12.98759842, 10.59689903,  4.9968996 , 17.38139725, 11.19999886,
        1.39999998, 19.19689941, 10.19689941, 12.3844986 ,  5.79379988,
       13.39689827,  5.        , 10.19689941,  3.19689989, 21.57519722,
       10.39379883, 14.59069824, 12.78759956,  2.19689989,  6.59069967,
       11.39379883, 17.38139725, 15.59069824, 17.39379883,  7.        ,
       35.78759766,  6.39379978, 11.39379883,  9.39379978,  8.19689941,
        8.19689941, 14.19689941,  3.59689999,  6.19689989, 10.18759918,
        6.39379978,  2.19689989, 11.19999886,  8.39999962, 11.5875988 ,
        5.        ,  3.19689989,  7.19689989, 12.39379883,  5.19689989,
       31.76899338,  9.39069939, 12.98759842, 26.        , 30.19689369,
        7.7968998 ,  6.19689989, 10.79069901,  4.9968996 ,  5.        ,
        6.59069967, 15.98139763, 20.19689941, 15.37829781, 12.3844986 ,
        2.19689989,  9.99379921, 16.19689941, 11.19689941, 10.        ,
       10.39379883,  2.19689989, 18.19689941,  7.7968998 , 21.38759613,
       11.39379883, 18.39379883,  5.        , 11.59070015, 11.19689941,
       26.59069824,  8.        , 20.78449631, 22.78759766,  9.59070015,
        7.1937995 , 48.36279297,  2.79999995,  6.19689989,  7.99069977,
       32.78139496,  4.39379978,  3.19689989, 16.18139839, 12.98449898,
       12.59999847, 10.19689941, 15.78759766, 11.5875988 , 13.59069824,
       16.9937973 ,  6.39379978,  8.78759956, 11.98449898,  5.39379978,
       12.59070015,  4.19689989,  8.78759956,  7.7968998 , 21.98449707,
       14.        ,  4.        ,  6.59069967,  7.1937995 ,  7.99069977,
       12.98759842,  8.39379978,  8.78759956, 11.19999886, 14.19689941,
        5.79379988, 11.98449898, 17.19689941, 28.77209473, 25.98449707,
       16.98449707, 21.77209663, 14.79689789,  7.7968998 ,  9.39069939,
        2.19689989,  4.9968996 ,  9.59070015, 10.59689903,  6.3968997 ,
       12.78759956, 16.9937973 , 11.19689941,  7.1937995 , 10.        ,
       39.78139114, 17.59689713, 15.39379883,  8.39379978, 11.99689865,
        9.19689941, 20.18139648,  3.59689999, 12.59069824, 12.19069862,
        4.39379978, 13.39689827, 19.78759766,  6.39379978,  7.59069967,
        5.19689989, 15.78759766, 64.95968628, 20.77209663,  6.19689989,  8.39379978,  7.99069977,  8.39379978,
        1.39999998, 10.39379978,  8.39999962, 17.98449707, 29.36279488,
       10.39379883,  2.        , 15.59070015, 14.38759804, 15.39379883,
       17.57519722,  2.19689989, 15.37829781,  3.19689989, 10.59689903,
       13.19689941,  6.39379978,  6.19689989,  3.59689999,  3.59689999,
        6.59069967,  7.39379978,  4.39379978, 12.79379845,  0.        ,
        4.19999981,  4.39379978, 18.18139839,  2.79999995,  6.19689989,
        4.39379978,  4.        , 10.59070015, 17.38139725,  9.99379921,
       14.38759804,  5.79379988,  7.99069977, 11.39379883, 17.57519722,
       11.59070015,  5.79379988,  8.78759956,  3.19689989, 12.3844986 ,
       10.59070015, 23.        , 30.78759956,  6.3968997 , 14.79689789,
        6.        ,  8.39379978, 12.19069862,  2.        , 10.79069901,
        2.        , 11.78759956,  2.19689989,  5.        , 13.59069824,
        6.        , 15.        , 13.78759956, 11.78759956, 15.        ,
       18.58759689, 11.59070015,  9.19689941,  1.        , 13.78449821,
       14.78759766, 16.58449745,  8.39379978,  7.        , 19.        ,
       17.19689941,  3.59689999,  7.1937995 ,  3.19689989,  2.79999995,
        7.19689989,  7.1937995 , 19.77209663, 11.5875988 , 16.9937973 ,
       12.19689941, 19.57519722, 17.39379883,  6.3968997 ,  3.        ,
        8.78759956,  9.59070015,  7.39379978,  4.39379978,  4.19999981,
        4.19689989,  6.        ,  5.19689989, 27.39379883,  7.        ,
       19.59069824,  7.59069967,  6.39379978,  3.        ,  6.39379978,
        7.59069967,  5.39379978,  9.39379978,  3.19689989,  7.7968998 ,
        2.19689989, 23.        ,  9.39379978,  9.78759956,  8.78759956,
        6.39379978,  7.        ,  2.19689989,  7.        ,  4.9968996 ,
       19.98449707,  8.59379959,  8.59379959,  3.        , 17.98449707,
       20.37829781,  9.19689941,  7.59069967, 14.18139839, 13.39379883,
        7.        , 20.78759766,  7.7968998 , 11.99689865, 17.18139839,
        5.5999999 ,  8.59070015,  5.79379988,  9.19689941,  8.59070015,
        4.19689989,  7.99069977, 11.39379883,  0.        , 10.18759918,
       11.78759956,  8.78759956, 11.5875988 ,  7.7968998 ,  3.19689989,
       10.98449898,  9.39379978, 13.39689827, 10.59689903, 30.56279373,
       17.38139725, 19.77209663,  7.        , 12.98449898,  7.59069967,
        4.19999981,  8.59070015,  2.79999995, 18.        ,  4.        ,
       13.39379883,  2.19689989, 4.9968996 ,  4.9968996 ,  7.39379978, 17.59689713,  5.39379978,
        2.19689989, 23.19689941, 12.98759842,  3.59689999,  5.19689989,
       12.98449898, 11.59070015,  8.        ,  7.7968998 , 12.39379883,
       10.79069901, 27.77209473, 15.78759766, 16.58449745, 10.98449898,
        5.19689989,  7.39379978,  9.39069939,  3.19689989,  5.19689989,
       11.39379883, 16.39379883, 29.37829399, 13.98449898, 10.19689941,
       18.39379692,  2.        ,  1.39999998, 37.7565918 ,  6.39379978,
       30.55969429, 23.19689941,  5.        , 12.79379845, 17.59069824,
       12.98759842, 32.3782959 ,  8.39379978, 12.3844986 ,  8.59070015,
        7.19689989,  3.19689989, 12.39379883, 12.98759842, 12.98759842,
        5.5999999 ,  5.39379978, 12.19689941,  2.79999995, 10.59070015,
       11.        , 20.78759766,  1.39999998,  4.        ,  6.3968997 ,
        3.59689999,  7.1937995 , 10.59070015, 13.39379883,  3.59689999,
       16.58449745,  5.        ,  7.1937995 , 22.3782959 ,  6.39379978,
        9.59070015, 10.39379883,  2.19689989, 22.18139648,  9.39069939,
        4.39379978, 18.        ,  3.19689989,  2.        , 10.78759956,
       11.98449898, 14.79689789,  2.19689989, 16.39379883, 13.39689827,
        7.19689989, 15.59379768, 10.98449898, 21.17209625,  1.        ,
       22.57519722, 15.78759766,  9.        , 13.59069824, 20.19689941,
       10.79069901,  7.        , 12.39379883,  2.19689989, 21.39379883, 10.79069901, 14.38759804,
       12.98759842, 10.79069901,  2.19689989,  8.59379959,  2.79999995,
        8.19689941, 17.78759766, 13.99999809, 10.59689903,  9.        ,
        8.        ,  4.        , 16.39069748, 16.18139839, 11.99689865,
        5.19689989,  2.        , 20.18139648, 12.78759956,  7.39379978,
       16.39379883, 14.18139839,  3.19689989,  2.19689989, 21.59070015,
        4.19999981,  7.19689989, 14.39379883, 17.59069824, 16.19689941,
       23.3782959 , 10.39379883,  4.19999981, 18.19689941,  3.59689999,
       13.39379883, 11.5875988 ,  3.19689989, 14.39379883,  3.19689989,
        3.19689989,  5.39379978, 12.19689941,  9.19689941, 17.78759766,
       21.96899605, 24.59689522, 29.3782959 , 16.98449707, 13.19689941,
       13.78449821, 10.98449898,  8.39999962, 13.19689941,  8.59379959,
        6.3968997 ,  2.19689989, 18.39379692,  4.        , 10.98449898,
        6.59069967,  8.39379978,  6.39379978,  3.19689989, 11.59070015,
       16.18139839, 11.19689941,  6.        ,  5.79379988,  3.19689989,
       13.18139839, 18.78759766, 16.39069748, 11.78759956,  8.39999962,
       18.37829781,  9.39069939,  6.3968997 , 21.77209663, 12.79379845,
        8.39379978,  9.78759956, 17.78759766, 12.        ,  8.59379959,
        6.        , 17.38139725, 16.19689751, 13.19689941, 17.59069824,
       25.17829514,  1.39999998, 15.39379883,  9.59070015, 12.19689941,
       18.58759689, 20.18139648,  6.3968997 ,  6.19689989, 22.3782959 ,
       15.98449898,  7.1937995 ,  3.59689999, 14.78759766, 26.78759766,
       11.99689865, 14.38759804, 11.5875988 , 13.18139839,  6.59069967,
        5.        , 16.19689941,  0.        ])
y = np.array([
       20.59069824, 14.19379807, 14.99069786,  5.        ,  3.59689999,
       11.19999886,  9.39069939,  9.99379921, 17.18139839, 30.17519379,
        8.19689941, 20.77209663, 52.98449707, 29.98139381, 18.18139839,
        2.19689989, 16.58449745, 11.78759956,  9.59070015, 13.78759956,
       23.18139839, 13.39689827, 25.39379883, 20.39689636, 10.59070015,
       41.59069443, 19.98759651,  6.        , 20.98449707, 17.18759727,
       15.18449783,  7.39379978, 13.39379883, 14.59069824,  2.        ,
       39.78139114, 26.77209473, 38.96279144, 17.98449707, 17.98449707,
        9.78759956, 21.19689941, 12.98759842, 22.3999958 , 16.39379883,
       10.59070015, 16.39069748,  9.79999924, 11.98449898, 27.97829437,
        7.7968998 ,  7.59069967, 12.78759956, 14.59069824, 15.39999771,
       41.56899261, 24.98449707, 29.98139381, 24.19689941, 42.39379883,
       26.98449707, 23.59069824, 19.38449669,  8.39379978,  5.39379978,
       16.59069824,  5.39379978, 12.98759842,  7.1937995 , 11.98449898,
       14.58139801, 11.78759956, 29.98449707,  4.19999981,  7.99069977,
       26.96899605, 83.57519531,  1.39999998,  8.59379959, 48.13798904,
       12.78759956, 60.15658569, 45.74108887, 26.3782959 ,  9.59070015,
       25.59069824, 23.3782959 , 13.98449898,  4.39379978, 26.9875946 ,
       17.59689713,  5.79379988, 15.59069824, 33.1689949 , 16.37829781,
       13.39379883,  5.19689989, 17.37829781, 14.99069786,  7.        ,
       25.37209511, 11.78759956, 11.99689865, 32.96899414,  9.99379921,
       26.        ,  8.19689941, 14.58139801, 12.98759842, 11.78759956,
       11.19999886, 11.59070015, 69.59069824, 14.18139839, 19.77209663,
       26.57829475, 18.39379692, 25.58759499, 25.58759499, 11.78759956,
       47.7503891 , 34.97829437, 20.39379883,  8.19689941, 15.98139763, 13.59069824,  9.        ,
       12.59999847, 30.55969429,  9.39069939, 11.99689865, 10.59070015,
       17.7906971 ,  6.3968997 ,  8.        ,  8.19689941, 25.59069824,
       29.98449707,  7.39379978, 19.37829781,  8.78759956,  6.59069967,
       28.17209435,  1.        ,  6.39379978, 25.96899605, 15.78759766,
        2.        ,  8.19689941,  5.39379978, 11.5875988 , 12.19069862,
        5.19689989,  7.        , 19.38449669, 18.18139839, 12.98449898,
        6.39379978,  4.9968996 , 17.59689713, 14.19379807,  8.        ,
        9.19689941, 24.59689522, 14.79689789, 34.55969238,  3.        ,
        9.79999924, 21.77519608, 45.34729004, 19.59999657,  5.79379988,
       13.78449821, 11.39379883,  5.        ,  5.19689989, 15.19689941,
       41.59069443, 13.        , 13.39379883,  1.        , 19.77209663,
        4.39379978,  6.        ,  5.5999999 ,  8.39379978, 16.37829781,
        8.59379959, 15.39379883, 18.39379692,  5.39379978,  1.39999998,
        6.39379978, 15.18139839,  8.78759956, 14.59069824, 13.18139839,
       12.19069862, 12.59069824, 10.39379883,  8.39379978,  8.19689941,
        5.5999999 , 29.18139648, 17.19689941,  1.        , 41.5472908 ,
       12.19069862, 50.55038834, 44.39379883, 25.98449707,  8.39379978,
       10.39379883,  6.39379978,  0.        , 10.18759918,  8.59070015,
       11.78759956, 14.98449898, 20.98449707, 22.78759575,41.56899261, 15.98139763,  3.59689999, 21.57519722, 12.79379845,
        5.19689989,  6.3968997 , 22.59379578,  6.3968997 , 25.19689941,
       20.18139648, 14.19689941, 14.39379883,  7.7968998 , 10.        ,
        7.7968998 ,  4.9968996 ,  7.19689989, 15.18139839,  5.39379978,
        2.19689989,  9.19689941,  6.59069967, 16.39379883, 16.19689941,
       18.98449707,  8.78759956,  7.1937995 , 10.19689941,  2.        ,
       10.79069901,  7.        , 16.79999733, 11.39379883, 12.79379845,
       11.        , 13.18139839, 17.39379883, 25.56589508, 18.59069824,
       34.7565918 , 41.34729004, 18.        , 21.5813961 , 17.57519722,
       29.77209473, 34.59069824, 14.99069786, 13.        ,  2.19689989,
       15.37829781,  9.59070015,  3.59689999,  1.        , 17.18759727,
       27.39689445,  8.19689941, 12.98759842,  3.19689989,  6.59069967,
        4.9968996 , 19.19069672, 12.59069824,  7.59069967,  6.39379978,
        3.19689989, 19.78759766, 11.5875988 , 23.17519569, 12.19689941,
       13.39379978, 15.18449783, 10.59070015, 23.57519531,  8.19689941,  7.        ,  9.59070015,
       10.18759918, 14.58139801, 10.98449898,  7.1937995 ,  4.        ,
        3.19689989,  8.59070015, 16.37829781,  6.19689989,  3.59689999,
       11.78759956, 16.58449745,  6.39379978,  6.39379978, 32.57519531,
       11.39379883,  9.39069939,  9.79999924, 16.78759766, 23.98449898,
        2.79999995, 31.57519722,  3.59689999, 13.59069824,  7.39379978,
       17.19689941, 31.39379883, 20.98449707, 14.59069824,  7.99069977,
       19.79379654, 22.57209587,  9.99379921,  9.99379921, 12.39379883,
       10.19689941, 13.59069824, 14.99069786, 28.96899414, 31.35969353,
        9.19689941, 21.38759613, 21.78759766, 24.57519531, 25.98449707,
       15.98449898, 18.19689941, 25.19999504, 29.98139381,  8.19689941,
       33.        , 22.96899605, 26.77209473,  6.39379978, 22.59379578,
       17.78759766, 17.98449707, 17.19689941, 10.18759918, 16.58449745,
       13.        ,  8.59379959, 10.59070015,  8.59070015, 21.5813961 ,
       16.39069748, 11.59070015, 10.18759918,  4.39379978, 19.19069672,
        9.39379978,  8.78759956, 23.18139648, 10.98449898, 14.19379807,
       15.59069824, 35.5813942 ,  9.19689941, 21.3782959 ,  4.        ,
       18.78759766,  7.        ,  2.        ,  6.59069967, 13.39689827,
        9.19689941, 13.78759956, 18.98449898, 17.98449707, 14.18139839,
       27.3782959 ,  7.99069977,  5.39379978,  7.99069977, 20.59069824,
        5.79379988,  2.19689989,  7.1937995 , 10.        , 11.5875988 ,
       17.98449707, 20.98449707,  5.19689989, 25.97519493, 23.1968956 ,
       43.95349121, 28.57519531,  4.39379978, 42.77209473, 12.19689941, 21.57519722, 18.19999695,  7.59069967,
        9.78759956,  6.19689989,  9.78759956, 18.78139687, 12.19689941,
       10.19689941, 14.19689941, 14.        , 35.77209473,  8.59379959,
       40.36279297,  7.7968998 ,  7.99069977, 10.98449898, 15.19689941,
       13.59070015, 10.19689941, 10.19689941, 12.78759956,  7.7968998 ,
       29.39379883,  9.19689941, 10.59070015, 24.        ,  4.19689989,
        2.19689989, 22.19689941,  5.        , 18.39379692, 22.98139572,
       24.38139534,  5.        ,  5.        ,  4.19689989,  1.        ,
       11.39379883, 13.39379883,  8.59070015, 13.78759956,  8.39379978,
       10.78759956, 20.37519646, 15.59069824, 11.98449898, 15.18449783,
        5.19689989,  3.19689989, 18.78759766,  9.39379978,  4.39379978,
       15.19689941, 11.39379883, 19.79379654, 14.38759804, 12.59069824,
       29.98449707, 17.19689941,  6.3968997 ,  2.79999995,  9.59070015,
       27.59069443, 17.57519722,  7.59069967,  5.79379988, 13.18139839,
       24.57519531,  8.39379978,  4.39379978,  3.19689989,  6.19689989,
       13.78449821,  6.59069967, 16.98449707,  7.1937995 ,  5.19689989,
       23.78759956,  4.9968996 , 18.39379692,  7.1937995 , 14.58139801,
       13.39379978,  7.1937995 , 12.3844986 ,  7.39379978,  9.78759956,
        8.39379978, 22.18139648, 22.96899605, 12.3844986 , 12.19689941,
        5.79379988,  7.1937995 ,  2.79999995, 22.3782959 ,  7.39379978, 48.96899414, 18.19999695, 14.39379883, 27.78759766, 13.18139839,
       33.78759766, 11.        , 14.99069786,  5.19689989,  8.59070015,
       15.98139763,  8.39999962, 13.18139839,  9.59070015, 10.19689941,
       15.98449898, 12.19069862, 17.78759956,  7.7968998 ,  6.        ,
       14.19689941, 16.39069748, 21.38759613, 24.        ,  4.39379978,
       16.39379883,  5.79379988,  8.19689941, 27.96899414,  2.19689989,
       10.19689941, 12.3844986 , 28.18139648,  5.        ,  6.39379978,
        9.19689941,  9.19689941,  5.        , 16.18139839,  9.19689941,
       12.19689941, 24.78759766,  8.78759956,  9.39379978,  5.19689989,
       11.19689941,  5.79379988,  2.19689989, 26.9658947 , 22.78759956,
       23.77829552,  6.39379978,  2.        ,  5.79379988,  6.19689989,
       12.98449898, 61.55969238,  4.        , 29.36279488, 20.37829781,
       25.37209511,  5.        ,  3.19689989, 21.18139648, 11.39379883,
       10.98449898, 14.18139839, 20.57519722, 10.19689941, 11.59070015,
       13.59069824,  6.59069967, 25.56589508, 16.18139839,  6.59069967,
       15.37829781, 50.54419327,  8.19689941, 17.7906971 , 39.95349121,
       21.5813961 , 12.78759956, 27.3782959 , 28.77519417,  5.79379988,
       25.59069824, 23.99379539, 16.59069824, 16.19689941, 17.18139839,
       11.39379883,  4.19689989,  9.59070015, 12.59069824, 10.19689941,
       12.19069862, 18.97519684, 16.39379883, 27.56899452, 19.19069672,
        3.19689989, 11.98449898,  6.        ,  4.19689989,  9.19689941,
        2.79999995, 27.96899414, 14.98449898, 13.78759956,  2.19689989,
       37.7565918 , 14.58139801, 14.99069786, 11.78759956, 11.98449898,
        8.19689941,  3.        , 13.59069824,  7.        ,  9.39379978,
        8.59070015, 10.98449898, 15.78759766,  3.19689989, 15.59070015,
        3.        , 20.39689636, 62.18140793, 10.59070015, 12.79379845,  9.39379978,  5.        , 13.78449821,
        7.1937995 ,  5.19689989, 22.57519722, 18.19999695, 22.57519722,
        9.78759956, 24.18139648, 16.19689751, 13.39379978, 39.78759766,
       17.57519722,  8.19689941, 33.36279297, 18.18139839,  8.59070015,
        7.99069977, 10.79069901, 15.59069824,  2.79999995, 11.19689941,
       20.37829781, 12.79379845, 11.        ,  3.        ,  2.19689989,
        5.39379978, 12.98449898, 11.39379883,  9.99379921,  6.59069967,
        4.        , 10.98449898, 11.78759956,  2.79999995, 18.39379692,
       11.99689865,  2.        , 12.3844986 , 10.19689941,  6.39379978,
        5.5999999 ,  8.59379959, 11.5875988 ,  1.39999998, 13.78449821,
       12.3844986 , 24.19689941, 29.37829399,  4.19689989,  6.39379978,
        1.        , 14.18139839, 21.5813961 , 17.98449898,  4.39379978,
        5.39379978,  9.39069939,  6.59069967,  9.99379921,  9.        ,
       16.77829742, 10.19689941,  7.19689989, 10.19689941, 25.58759499,
       23.97209549,  2.        ,  5.19689989,  7.19689989, 20.57519722,
        5.5999999 , 10.19689941,  2.19689989,  7.99069977,  8.39379978,
        7.        , 11.19689941, 23.57519531,  3.19689989, 16.19689941,
       10.79069901, 14.58139801, 11.99689865, 19.19689941, 13.98449898,
       18.        ,  2.19689989,  4.        , 11.78759956,  2.        ,
       11.19689941, 10.98449898, 17.38139725,  5.5999999 , 12.59069824,
       11.99689865,  5.19689989, 34.57519531, 40.96589279,  7.99069977,
       15.98139763, 18.37829781,  5.39379978, 44.3782959 ,  4.9968996 ,
       20.57519722,  4.9968996 , 28.37829781,  5.39379978, 21.78759956,
        2.19689989, 18.78139687, 14.79689789,  6.19689989, 23.17519569,
        8.        , 12.79379845,  7.19689989, 13.78759956,  6.39379978,
        4.19689989,  7.99069977, 10.59689903,  8.39379978,  1.39999998,
        4.9968996 ,  6.39379978,  3.59689999,  3.59689999,  6.19689989,
       22.3782959 , 13.39689827, 15.18449783,  7.        ,  8.39999962,
       12.59999847,  5.19689989, 26.98449898,  1.39999998, 11.78759956,
        9.78759956,  7.7968998 , 26.3782959 , 22.3999958 , 15.18139839,
        8.59379959, 25.16589546,  3.59689999,  3.19689989, 12.        ,
       15.18139839, 17.19689941,  6.39379978,  6.        ,  4.        ,
       24.18139648, 19.37829781, 22.57209587,  9.39379978,  7.19689989,
       26.98449707, 23.36899567, 18.37829781, 14.58139801, 32.3720932 ,
       12.3844986 , 18.99689674, 31.18139648,  3.59689999, 18.37829781,  3.19689989,  6.19689989,
       12.78759956, 18.78759766,  3.19689989,  9.19689941, 20.77209663,
       10.98449898,  7.1937995 , 10.98449898, 16.19689751, 19.79379654,
       14.19689941,  2.19689989, 16.18139839, 18.57519722,  2.        ,
        4.39379978,  6.3968997 , 15.59069824, 14.78759766, 17.59069824,
       31.77209473,  7.99069977, 16.59069824, 15.98449898, 15.39379883,
       10.59070015, 41.59069824, 15.78759766, 17.78759766,  3.        ,
       10.98449898, 11.78759956,  8.59379959, 18.58759689, 10.18759918,
       11.5875988 ,  7.59069967,  5.79379988, 11.98449898,  2.19689989,
        7.59069967, 18.39379692, 14.59069824, 19.57519722, 31.55969429,
        8.19689941, 12.79379845, 21.17209625, 12.39379978, 19.59999657,
        9.39379978,  9.78759956,  8.78759956, 10.18759918,  8.59379959,
        7.19689989,  4.39379978, 33.1689949 , 16.9937973 , 10.98449898,
        8.59379959,  7.99069977, 17.38139725, 20.19689941,  6.19689989,
       16.78759766, 10.79069901,  8.        , 11.19999886,  8.19689941,
        5.        ,  5.39379978,  8.19689941,  6.39379978, 13.98449898,
        9.19689941,  6.19689989,  5.39379978, 19.18139648, 17.78759766,
        4.19999981, 21.19379616, 10.19689941,  5.39379978, 14.19689941,
       25.96899605,  4.        ,  8.        , 20.77209663,  8.59070015,
        6.39379978, 11.19999886, 7.        ,  3.19689989,  8.39379978, 24.59069824,  7.39379978,
        7.        ,  3.59689999, 41.39379883,  2.79999995,  7.19689989,
        4.9968996 ,  4.39379978, 12.98759842, 34.15969467,  3.59689999,
       28.96899414, 10.98449898, 31.96899414, 26.77209473,  4.9968996 ,
        3.19689989, 31.35969353, 13.        , 10.79069901,  9.39379978,
        4.9968996 ,  9.19689941,  8.59379959, 15.39379883, 14.58139801,
        8.78759956,  7.19689989, 12.39379883, 16.9937973 , 13.39689827,
       19.39379883, 23.39069557, 43.35659027, 14.18139839,  3.59689999,
        9.39379978, 21.18139648, 12.59069824,  7.59069967, 10.59070015,
       11.5875988 , 10.98449898, 11.99689865, 11.19999886,  6.39379978,
        9.39069939, 24.19689941, 13.98449898, 19.78759766, 11.        ,
       11.98449898, 29.78759384, 25.99689484,  6.59069967,  8.59070015,
       11.19689941,  8.39999962, 39.19689941, 51.16589355, 15.78759766,
        7.        ,  9.19689941, 24.16589546,  8.        , 29.36279488,
       44.97209167, 49.15039062, 47.59069824, 37.19689941, 27.16589546,
       16.58449745, 12.59069824,  3.19689989,  9.19689941,  8.59379959,
        5.        , 14.39379883, 31.38139343, 18.17829704,  9.99379921,
       12.78759956, 13.19689941, 25.97519493, 23.98449707,  2.        ,
       14.18139839, 20.37829781, 18.58759689, 12.98759842,  7.19689989,
       11.39379883,  7.99069977, 16.18139839, 10.        ,  7.59069967,
       14.19379807,  3.59689999, 11.19689941,  5.79379988, 10.19689941,
       10.18759918,  3.59689999,  3.59689999,  2.        , 14.        ,
       10.98449898,  5.39379978, 20.18139648, 11.19689941,  2.79999995,
       12.78759956,  5.19689989, 38.35969162])

from scipy.stats import kstest
kstest(x, y)

KstestResult(statistic=0.10871694417238002, pvalue=1.1281169193083454e-05)

In [161]:
np.size(x)

1021

In [162]:
np.size(y)

1021

In [163]:
x_off = np.array([58.51938629, 37.1968956 , 28.18139648, 39.3720932 , 12.39379883,
       30.55969429, 26.38449478, 19.19689941, 31.16589355, 28.79689407,
       17.98449707, 20.97829628, 26.9658947 ,  6.3968997 , 28.78759766,
       11.19689941, 13.78759956,  8.39379978, 23.99379539, 27.78759956,
       34.3782959 , 23.77209663, 21.39379883, 29.55969429, 36.59379196,
       29.59379387, 16.        , 14.59069824, 38.76899338, 11.59070015,
       31.39379883, 22.        , 20.37829781, 22.59069824, 29.57209396,
       32.57519531, 13.39689827, 24.39379883, 29.76589394, 44.74108887,
       19.59069824,  8.39379978, 18.59069824,  7.7968998 ,  7.19689989,
       28.96899414, 21.39379883,  9.59070015, 41.57519531, 14.19689941,
        7.59069967, 37.7565918 , 11.5875988 ,  7.39379978, 30.55969429,
       15.78759766, 26.57519531, 54.57829666, 58.39071274, 23.18139648,
       19.18139648,  8.78759956,  7.        , 25.96899605,  6.19689989,
       18.18139839,  8.        , 14.79689789,  8.78759956, 10.59070015,
        6.        , 23.78759766,  6.3968997 , 20.97829628, 14.19379807,
       17.78759956, 13.18139839, 19.37829781, 13.59069824, 12.98759842,
       47.38449478,  4.19999981, 21.78759766, 15.19689941,  9.59070015,
       31.35969353, 77.106987  , 67.77209473, 17.98449707, 55.15039444,
       18.59069824, 16.77829742,  9.19689941, 35.36589432, 13.78759956,
       35.19379425, 14.99069786, 17.        , 14.38759804, 59.78759766,
       21.17209625, 46.7565918 , 22.59379578, 39.75969315,  2.19689989,
       26.19069481, 23.19689941, 27.77209473, 25.98449707, 31.78759766,
       33.38449478, 36.78759766, 21.38759613, 41.59069824,  5.79379988,
       67.78759766,  5.19689989, 19.38449669, 44.15039062, 18.99689674,
        2.        , 27.96899605, 20.18139839, 41.59069824, 26.16589546,
       40.19689941, 32.55969238, 14.99069786, 18.78139687, 19.18139648,
        6.19689989, 38.95349121, 12.39379883,  7.1937995 , 34.78449249,
       23.77209663, 12.78759956,  9.78759956,  6.59069967, 14.38759804,
       29.18449402, 20.37829781,  1.        , 23.39379883, 27.78759766,
       34.95349121, 16.59069824, 15.        , 29.57209396, 41.59069824,
       13.59069824, 46.78759766, 16.78759956, 44.96899414, 26.78759766,
       11.        , 21.78759766, 31.57519531, 26.59069824, 27.57519531,
       13.18139839, 11.        , 46.77209473,  3.19689989, 18.78759766,
       12.78759956, 41.19689941, 18.37829781, 38.18759155, 31.36279488,
       13.18139839,  8.39999962,  4.19689989, 68.79382324, 14.19379807,
       12.78759956, 45.79069901, 20.78449631, 20.37519646, 92.48218536,
       86.57519531, 57.35658646, 53.98449707, 2.        , 21.38759613, 21.77519608, 12.19689941, 10.98449898,
       13.78759956, 29.        ,  5.        ,  7.1937995 , 17.18759727,
        5.5999999 , 31.78759766,  8.59070015, 13.59069824, 25.17829514,
       26.36279488,  8.19689941, 29.        , 37.78759766, 25.58759499,
       28.57519531, 33.59999466, 15.39379883, 16.77829742,  7.39379978,
        6.59069967, 33.57829285, 40.98759079, 12.19689941, 41.96899414,
       23.19689941, 14.19379807, 18.        , 23.57519531, 21.19689941,
       32.58759308, 35.19379425, 16.18139839, 23.36899567, 19.78759766,
       16.39379883, 40.7565918 , 14.19379807,  9.78759956,  4.        ,
       13.39689827, 11.39379883, 18.97519684, 16.58449745, 33.77209473,
        4.39379978, 27.97829437,  5.79379988, 19.78759766, 11.19689941,
       22.77209663,  9.        , 27.39379883, 21.96899605, 32.39379501,
       33.1689949 ,  8.39999962, 21.79689598,  1.39999998, 26.77209473,
       15.78759766, 19.        , 28.19379425,  9.39379978,  1.39999998,
       45.33798981, 16.        , 17.39379883, 14.99069786, 14.19379807,
        2.19689989,  9.78759956, 17.59689713, 16.9937973 , 22.57209587,
        5.79379988, 23.79999542, 14.59069824, 32.95349503, 15.39999771,
       12.78759956,  3.59689999, 12.78759956, 16.37829781, 33.55969238,
       38.3782959 , 35.3782959 ,  3.59689999, 46.3782959 , 52.36279297,
       46.99690628, 31.36279488, 23.98449707, 29.59379387, 96.49768066,
       26.        , 45.35969162, 34.        , 37.16589355, 71.11938477,
       58.18139648, 52.33178711, 28.36279488, 31.18759346, 12.39379883,
       17.38139725, 17.18759727, 13.39379883, 14.39379883, 48.97829056,
        6.39379978, 18.98449898, 37.3782959 , 13.59069824, 13.59069824,
       11.39379883, 11.5875988 , 44.59069824, 28.39379883, 44.34729385,
       30.98449707, 25.59069824, 54.7565918 , 16.9937973 , 19.38449669,
       27.16589546, 36.57209396, 15.        ,  22.3999958 ,  56.19381332,  29.16279411,
        44.57519531,  10.39379883,   9.39379978,  46.39379883,
         4.39379978,   9.59070015,   1.39999998,  85.35660553,
        47.95349121,  56.9255867 ,  35.5813942 ,  19.77209663,
       117.77209473,  45.57519531,   2.19689989,   8.78759956,
        48.15969086,  10.39379883,  43.57209015,  24.38139534,
        11.39379883,  46.73799133,   7.        ,  17.98449707,
        22.59379578,  22.57209587,  30.56279373,  30.        ,
        18.98449707,  11.98449898,   8.59070015,  26.19069481,
        31.96279335,  12.19069862,   4.19689989,   7.19689989,
        11.39379883,  28.17209435,  14.39379883,   5.39379978,
         6.19689989,   3.        ,   8.59070015,   5.5999999 ,
        45.35969162,  30.59069824,  17.18759727,  56.55968857,
         8.19689941,   9.39069939,  12.59069824,  30.19689369,
        13.78449821,  30.58449364,   8.59070015,  24.38139534,
         6.19689989,  16.98449707,  12.59069824,  17.59689713,
        17.39379883,  27.37519455,  13.19689941,  50.57519531,
         7.19689989,  54.95349121,  46.74108887,  39.15659332,
        45.        ,  46.98449707,  46.78139114,  56.34729004,
        55.36279297,  25.37209511,  53.97519302,  60.15658569,
        37.57519531,  14.99069786,   7.7968998 ,   3.        ,
        30.19689941,  38.76899338, 15.18139839, 25.58759499, 30.58449364,  5.        , 11.98449898,
       22.77209663,  5.79379988,  3.19689989, 25.18139648, 30.39379883,
       21.39379883, 12.78759956,  6.39379978, 20.18139648, 10.78759956,
       22.        , 13.59069824, 31.18139648, 23.36899567,  9.59070015,
       21.99069595,  9.78759956, 14.99069786,  9.99379921, 28.96899414,
       21.5813961 , 18.39379692, 20.37519646, 13.39379883, 41.56899261,
        8.39999962, 10.18759918,  3.        ,  3.19689989, 13.19689941,
        7.1937995 , 11.39379883, 27.56899452, 27.59070015, 48.18139267,
       13.98449898, 30.36279488, 34.97829437, 18.98449707, 38.3782959 ,
       35.96899414, 32.96899414, 22.57209587,  5.79379988, 27.97829437,
       17.98449707, 22.57519722, 16.37829781,  7.7968998 , 14.58139801,
       63.93798828, 15.78759766, 18.39379692, 19.78759766, 30.56279373,
        2.79999995, 10.59689903, 28.59069824, 18.78759766,  2.        ,
       19.59070015, 15.59069824,  2.19689989, 38.18139648, 29.57209396,
       39.36279297,  3.59689999, 19.79379654, 41.57519531, 39.75969315,
       17.57519722,  6.59069967, 15.78759766, 28.79689407, 21.3782959 ,
       26.98449707, 21.57519722, 20.19689941, 17.        , 38.77209473,
       34.56899261, 28.77519417, 18.19689941, 37.7565918 , 25.3782959 ,
       18.78759766, 33.36279297, 10.98449898, 20.39379883, 26.19689941,
       29.39379883, 63.39379883,  8.59070015,  6.59069967,  8.78759956,
       18.97519684, 11.19689941, 25.78759766, 13.98449898, 30.18139648,
       35.3782959 , 38.18139648, 45.98449707, 46.19689941, 28.77209473,
        7.19689989, 24.16589546, 22.18449593,  5.79379988, 40.94419098,
       27.77209473, 21.96899605, 13.59069824, 18.99689674, 36.39379883,
        7.99069977, 19.79379654, 18.78139687, 30.78759766, 34.35349274,
       86.98449707, 52.76898956, 29.57209396, 68.77210236, 87.51007843, 19.98449707, 10.59070015, 12.        , 17.        , 31.35969353,
       10.98449898, 18.18139839,  7.        , 10.39379883, 44.17518997,
       47.34729004, 22.77209663, 17.59689713, 20.59069824, 45.18139648,
       46.16589355, 32.3782959 , 28.39379883, 45.18139648, 77.55969238,
       55.35348892, 56.98449707, 73.89768219, 72.19689941, 73.1007843 ,
       80.57519531, 60.75968552, 43.74109268, 11.        ,  3.19689989,
       35.98449707, 25.37829781, 31.57519722, 19.77209663, 52.3782959 ,
       28.79689407, 21.77209663, 12.        , 31.35969353,  5.19689989,
       21.38759613, 36.3782959 , 52.96899414, 24.18139648, 32.16589355,
       28.77209473, 24.18759537, 28.55969429, 32.19689941,  8.78759956,
       36.15039444,  3.59689999, 22.39379883, 43.74109268, 16.58449745,
       39.7565918 , 15.59379768, 52.16589355, 49.15039062, 37.57519531,
        5.19689989, 19.39379883, 14.19379807,  8.19689941, 11.78759956,
       29.36279488, 13.59069824, 18.97519684, 22.98139572, 16.37829781,
       25.58759499, 26.38449478, 22.59069824, 32.59069824, 21.59069824,
       38.55969238, 24.18139648,  8.78759956, 35.19689941,  9.99379921,
       13.78449821, 29.77209473, 15.37829781, 37.78759766, 20.18139648, 12.3844986 , 10.59070015, 16.59069824,  7.19689989,
       10.79069901, 37.3782959 ,  7.59069967, 12.39379883, 23.98449707,
       63.3782959 , 13.98449898, 16.98449707,  7.7968998 ,  6.39379978,
       15.39999771,  5.39379978, 21.99069595, 10.98449898,  8.59070015,
       17.57519722,  3.19689989, 13.98449898,  5.19689989, 29.76589394,
       29.96899414, 34.99999619, 34.16589355,  4.39379978, 26.18139839,
       37.16589355, 10.79069901, 27.96899414,  8.78759956, 37.57519531,
       76.74108887, 29.36279488, 34.99999619, 62.3782959 , 56.59069824,
       72.95349121, 46.15039062, 54.98760605,  2.        , 13.59070015,
       49.96899414, 32.16589355, 44.19689941, 32.15659332, 18.99689674,
        6.        , 17.19689941, 12.98449898,  7.1937995 , 20.77209663,
       31.36279488,  8.78759956,  6.39379978,  7.59069967, 12.19689941,
       18.78759766,  3.19689989, 19.78759766, 11.39379883, 17.38139725,
       19.78759766,  9.19689941,  6.19689989, 23.77209663, 37.34729385,
       23.19689941, 58.77830124, 45.19689941, 47.94419098, 44.98449707,
       60.99692154, 52.14418793, 77.36590576,  4.        , 45.55969238,
       32.16589355, 48.59069824, 21.57519722, 27.18139458, 18.39379883,
       49.34729004, 40.36279297, 44.34729004, 47.18139648, 33.36279297,
       22.3999958 , 10.78759956,  7.19689989, 10.78759956,  7.1937995 ,
       26.98449898, 28.36279488, 35.19689941, 19.98759651, 21.96899605,
        5.39379978, 10.39379883,  4.39379978, 10.18759918, 18.18139839,
       20.59069824, 11.98449898, 20.59069824, 12.78759956, 16.39069748,
        5.19689989, 39.16589355, 46.56589127, 24.57519531, 51.1348877 ,
       24.76899529, 14.58139801, 28.77519417,  9.39069939, 29.78759384,
       26.98449707, 23.59069824,  2.        , 6.3968997 , 16.98449707,  4.39379978,  5.19689989,  8.59070015,
       18.39379692, 26.39379883, 12.59069824, 43.36279297,  7.19689989,
       43.18139648, 18.39379692, 19.59999657, 11.39379883, 33.7565918 ,
       13.78759956,  6.39379978,  8.78759956, 10.18759918, 15.78759956,
       23.58449554, 51.13179016, 42.16589355, 40.55659103, 43.77209473,
       23.36899567, 19.        , 28.99069405, 30.55969429,  4.9968996 ,
        8.        ,  6.        , 23.39069557, 23.57519531,  8.39379978,
       15.37829781,  9.39069939,  5.        , 22.3999958 ,  7.39379978,
       20.39689636, 27.7844944 ,  4.9968996 ,  4.19999981, 19.18139648,
       13.        , 13.        ,  4.19689989, 44.19690323,  1.39999998,
       21.17209625, 13.39689827, 24.3782959 , 10.79069901, 11.5875988 ,
       18.39379883, 17.37829781, 21.39379883, 67.56589508, 55.39691544,
       13.        , 14.99069786, 14.78759766, 40.77209091, 60.32868576,
       10.19689941,  5.39379978,  6.59069967, 29.78759384, 29.95969391,
       19.77209663, 30.57519531, 17.7906971 , 28.55969429, 26.59069824,
        9.99379921,  4.39379978, 17.18139839, 52.76898956, 15.98139763,
        2.        , 48.95658875, 20.19689941, 14.19689941, 16.78759766,
       10.78759956,  5.79379988, 43.59379959, 43.57209015, 28.77519417,
        9.78759956, 23.39379883, 12.19069862, 15.78759766, 48.77209473,
       20.59069633,  5.79379988, 63.59069824,  7.39379978,  6.3968997 ,
        5.19689989, 20.98449707, 18.18139839, 31.36279488, 28.39379883,
       11.98449898, 21.19689941,  4.39379978, 38.96899414, 35.78759766,
       41.76279068, 61.1689949 , 15.78759766, 61.57830429, 42.15039062,
        4.19689989,  7.7968998 , 11.19999886, 12.79379845,  6.59069967,
        6.3968997 , 14.78759766, 32.7565918 , 13.59069824,  5.        ,
       33.1689949 , 10.18759918,  7.1937995 , 10.18759918, 23.98449707,
       19.18139648, 16.58449745, 55.        , 63.99071884,  30.39069366,  19.18139648,  90.54418945,
       142.3782959 ,   5.        ,  90.        , 141.39379883,
       125.72558594, 147.10388184,  66.91938782,  94.48218536,
       108.66047668,  71.57210541,  79.90698242,  55.5689888 ,
        90.90698242,  91.36279297,  14.59069824, 156.77824402,
        19.39379883,  37.16589355, 196.79669189, 149.31628418,
       197.5718689 ,  61.18139648, 158.01086426,  43.39379883,
       140.63256836,  13.78449821,  97.69458008,  24.18139839,
        42.58139038,  93.47597504,  18.58759689,  30.39069366,
        22.77209663,  64.16589355,  59.34729004,  22.3782959 ,
        45.15039062,  48.5689888 ,  72.92248535,  61.79381943,
        51.96899414,   6.3968997 ,  55.39379883,  46.16589355,
        43.76589203,  18.19999695,  17.39379883,   2.79999995,
        20.97829628,  20.57519722,  39.3720932 ,  26.16589546,
        45.5751915 ,  48.97829056,  37.77209473, 114.26047516,
       123.39388275,  47.15039062,  99.70077515,  47.7503891 ,
        72.90698242,  42.57519531,  51.95349121,  72.97210693,
        59.72558594,  76.71938324,  74.96899414,   7.19689989,
        71.10388947,  58.18139648,  41.57519531,  63.36589432,
        57.35658646,  22.78759766,  28.5813942 ,  13.59069824,
        11.78759956,   4.9968996 ,  47.59069824, 51.57519531, 79.57519531, 14.39379883, 78.72558594, 15.37829781,
       29.36279488, 56.98449707, 23.39069557, 42.3441925 , 68.31938934,
       38.59069824, 41.3782959 , 15.18449783, 15.18139839, 15.19689941,
       12.98759842, 22.18449593,  9.99379921, 16.18139839, 11.39379883,
       18.17829704, 58.7565918 , 59.1348877 , 49.96899414, 61.54418945,
       56.15039062, 20.39689636, 30.55969429, 69.78451538, 19.39379883,
       89.19074249, 20.97829628,  9.59070015, 14.98449898, 24.78759766,
       23.39069557, 30.58449364, 28.16589546, 56.1348877 , 24.57519531,
       35.16589355, 11.59070015, 42.        , 20.78759766, 25.58759499,
       12.79379845, 43.95349121,  4.39379978, 27.78759956, 77.7565918 ,
       42.34729004, 20.19689941,  8.19689941, 40.15039444, 31.59069824,
       43.40000534, 22.57209587, 24.78759766,  9.78759956, 41.54418945,
       41.19689941, 36.7565918 , 15.37829781,  1.39999998, 29.19689941,
        7.59069967, 16.9937973 ,  4.39379978, 22.18139648,  4.19689989,
       21.5813961 ,  7.99069977, 26.77209473, 18.17829704, 28.16589546,
       18.58759689, 21.19689941, 48.59070206, 28.38759422,  1.39999998,
       16.19689751, 47.55969238,  4.39379978, 33.77209473, 21.77519608,
       29.36279488, 17.98449707, 56.34729004, 51.34729004, 32.36279297,
       30.39379883, 38.3782959 , 68.12558746, 21.17209625, 66.90698242,
       21.99069595, 20.99999619, 42.15039062, 44.57519531, 28.38759422,
       15.18139839, 11.        , 10.59689903, 61.36279297, 30.18139648])
y_off = np.array([36.39379883,  9.39379978, 23.39379883, 42.00000381, 20.97829628,
       32.78139496, 20.77209663,  3.        , 36.7565918 , 27.76279449,
       11.19689941,  4.39379978, 17.37829781, 13.19689941, 28.38759422,
       36.57519531,  4.        , 39.75969315, 15.98139763, 24.79069519,
        5.79379988, 22.18139648, 16.39069748, 11.59070015, 11.39379883,
       11.19689941, 27.37519455,  8.59070015, 19.57519722, 31.18759346,
        9.39379978, 15.19689941, 14.38759804,  5.19689989, 27.78759766,
       36.57209396, 22.3782959 , 21.77519608, 17.38139725, 36.57519531,
       11.99689865, 18.19999695, 38.57519531, 31.59069824, 18.59070015,
        5.5999999 , 27.18139648, 17.19689941, 38.5534935 , 14.38759804,
        3.19689989, 35.5813942 ,  3.        ,  8.19689941, 18.78759956,
        4.19689989, 10.19689941, 58.36899185, 63.53798676, 21.98449707,
       21.17209625, 10.59689903, 17.98449707, 26.18139648, 13.98449898,
       19.78759766, 26.9658947 , 34.37519455, 11.5875988 ,  8.19689941,
       11.19999886, 35.57519531,  3.59689999, 15.59069824,  7.59069967,
       11.19689941, 25.98449707, 19.59999657, 23.58449554, 35.16589355,
       69.11938477, 15.78759766, 43.16279221, 35.36589432, 19.18139648,
        8.59070015, 69.16589355, 56.3782959 , 16.18139839, 10.78759956,
       19.77209663,  3.19689989, 21.19689941, 12.59999847,  9.19689941,
        6.3968997 , 15.59069824,  9.39379978,  4.19689989, 68.39379883,
       28.78759766, 54.57519531, 16.59069824, 45.95349121,  2.        ,
       29.78759384, 26.59069824, 45.79069901, 28.59069824, 45.59690475,
        8.59070015, 40.18139648, 78.72248077, 49.34418869, 13.18139839,
        2.79999995, 18.39379692, 26.77209473, 22.57209587, 22.57209587,
       58.19689941, 57.11938858, 55.54418945, 12.98759842, 55.72558594,
       10.59070015, 13.        , 47.57519531, 30.39379883, 40.94419098,
       51.58449936, 12.3844986 , 29.98139381, 38.38139343,  3.59689999,
       22.39379883,  9.99379921, 27.97829437, 17.78759766, 21.38759613,
       13.59069824, 31.77209473, 26.3782959 , 21.39379883, 18.57519722,
       16.19689751, 21.3782959 , 23.97209549, 22.19689941,  7.59069967,
       18.37829781,  8.39379978, 52.16588974,  2.19689989, 16.78759766,
       32.57519531, 17.        , 12.78759956, 22.19689941, 39.35039139,
       23.77829552, 30.79999352, 40.1906929 , 22.78759575, 41.15039062,
       41.57519531,  3.19689989, 29.16589546, 11.78759956,  9.99379921,
        3.        ,  6.59069967,  8.39379978, 37.16589355, 23.36899567,
       25.57519531,  3.59689999, 31.36279488, 31.96899414, 15.19689941,
        4.9968996 , 29.37829399, 23.78759766, 56.94728851, 23.39069557, 42.94729233, 13.19689941, 39.39379501,
       11.98449898,  6.3968997 ,  3.19689989, 37.56279373, 33.79379272,
       20.57519722, 11.99689865,  9.19689941,  7.39379978,  7.7968998 ,
       11.39379883, 17.        , 23.58449554, 26.57829475, 29.59069824,
       38.57519531, 39.54419327, 53.75968933, 41.15969086, 41.39690018,
       55.74108887,  7.7968998 ,  4.39379978, 18.17829704, 37.3782959 ,
       21.57519722, 31.78759766, 18.17829704, 36.39379883,  5.39379978,
       22.19689941, 72.31628418, 44.58449173, 31.55969429, 46.16589355,
       23.18139648, 70.19689941, 31.59069824, 31.78759766, 30.16589355,
       37.34729385, 51.3782959 ,  3.19689989, 18.39379883,  3.59689999,
       18.19689941,  7.        , 26.9875946 , 44.18139648, 16.39069748,
       36.78759766, 15.37829781, 26.18139648, 12.3844986 , 26.57519722,
       20.37519646, 18.19689941, 30.55969429, 26.18139648, 31.98449707,
       26.59999466, 16.58449745, 10.79069901, 29.36279488, 50.18139648,
        5.39379978, 40.39379883, 21.78759766, 43.1844902 , 29.96899414,
       61.38451004, 46.97518921, 63.32248688, 56.92248535, 33.39379883,
       29.18449402, 56.52868652, 29.16589546, 12.98759842, 56.38760757,
       52.96278763, 44.58449173, 10.        ,  7.59069967, 13.78759956,
        1.39999998, 30.19689369, 36.57519531, 38.96279144,  7.7968998 ,
       21.37829781, 43.39379883, 45.15039062, 35.98449707, 36.78759766,
       27.36279488, 22.57209587, 25.78759766, 14.59069824,  8.19689941,
        9.59070015, 14.78759766, 23.96899605, 27.18139458, 36.95349121,
       13.98449898, 20.78449631, 30.7565937 , 26.9658947 , 39.        ,
        3.19689989, 20.18139648, 22.3782959 , 16.77829742, 11.59070015,
       18.17829704,  2.        , 10.18759918, 19.59999657,  9.39069939,
       34.35349274, 18.37829781, 10.59689903, 36.3782959 , 26.19689941,
        8.59379959,  8.59379959, 19.19689941, 17.        , 31.18139648, 43.7565918 , 19.59070015,
       64.74108887, 55.3782959 , 64.16278839, 29.98139381, 26.59999466,
       15.78759766,  2.19689989, 49.93798828, 41.57519531, 22.59379578,
       36.55969238, 23.77829552, 19.78759766, 22.57519722, 31.59069824,
       18.58759689, 26.96899605,  5.        , 55.92248917, 70.71008301,
       23.99379539, 28.18139648, 16.19689751, 12.78759956, 25.39379883,
        7.        , 11.39379883,  8.59070015, 54.14728928, 29.78759766,
       77.106987  , 81.13488007, 29.19689941, 31.59689331, 23.36899567,
       47.77209473,  9.19689941, 39.39379501, 36.55969238, 23.99379539,
       22.59379578, 40.14729309, 26.38449478, 52.18139648, 16.19689941,
       31.39379883, 59.39379883, 20.59069633, 16.39069748, 64.92248535,
       57.19689941, 18.37829781, 60.39379883, 36.78759384, 27.59069824,
       42.38759232, 11.98449898, 49.79690933, 38.16589355, 28.98449707,
       35.96899414, 55.59069824,  4.19689989, 24.18139648, 10.59689903,
       17.18139839, 40.36279297, 28.57519531, 48.95349121, 15.18449783,
       50.18139648, 11.98449898, 24.98449707, 43.37829208, 54.57829666,
       12.78759956, 26.9875946 , 29.37829399, 36.57209396, 19.        ,
       26.57829475, 20.97829628,  22.78759766,  14.59069824,  10.18759918,
        31.79069328,  40.75039291,   2.19689989,   9.39069939,
        10.79069901,  24.57519531,  19.59069824,   8.19689941,
        14.38759804,   6.19689989,  22.78759766,  34.98449707,
         6.59069967,  11.39379883,  18.99689674,  29.57209396,
        20.39689636,  32.77209473,  28.36279488,  12.39379883,
        37.        ,  30.19689941,  31.16589355,  32.3782959 ,
         8.59070015,  56.19689941,  39.35039139,  18.78139687,
        45.35969162,  45.19689941,  44.36279297,  36.55969238,
         6.        ,  14.78759766,  16.18139839,  54.60001755,
        34.3782959 ,  38.7565918 ,  35.79689407,  27.3782959 ,
        57.72558975,  23.99379539,  16.77829742,  58.3782959 ,
         5.79379988,   8.19689941,   7.39379978,   1.        ,
        26.79379463,  50.93798828,  21.18139648,   7.39379978,
        19.37829781,  31.16589355,  13.19689941,  31.98449707,
         8.78759956,   2.19689989,  42.39379883,  35.77519226,
         7.7968998 ,  21.19379616,  37.96899414,  46.56589127,
        37.7565918 ,  19.19689941,   7.1937995 ,  10.18759918,
         9.39379978,   3.        ,  34.3782959 ,  48.7565918 ,
        27.18139458,  23.58449554,  38.18139648,   7.59069967,
        32.96899414,  13.98449898,  14.78759766,  23.79999542,
        14.18139839,  21.19689941,  16.37829781,  42.15039062,
        50.54418945,  64.        ,  44.54109192,  76.93798828,
        51.55969238,  11.39379883,  32.96899414,  29.3782959 ,
        24.79069519,  91.36592102,  27.59069824,  96.94420624,
        43.14109039, 109.77209473,  26.18139648,  32.75969315,
        63.33179092,  65.31628418,  65.11008453,  77.15039062,
        14.38759804,  13.78449821, 100.51318359,  44.36279297,
       100.89147949,   8.59379959,  48.74108887,  32.17829514,
        39.20000076,  30.36899376,  31.79069328,  17.19689941,
        35.3782959 ,  11.99689865,  15.37829781,  28.3782959 ,
        19.38449669,  27.7844944 ,  32.        ,  69.78451538,
        41.57519531,  44.7565918 , 23.18139648, 23.1968956 , 30.77829361, 20.77209663, 10.39379883,
       22.77209663, 46.54418945, 11.19999886, 13.39379883, 11.39379883,
       40.98759079, 60.96899414, 69.33178711, 55.13798904, 51.96899414,
       31.18759346, 35.55969238, 33.57519531, 41.57519531, 45.16588974,
       45.57519531, 31.57519341, 59.95349121, 37.15039062, 18.58759689,
       34.18139648, 20.78449631, 48.57519531, 49.00001144, 46.36279297,
       42.3441925 , 46.77209473, 48.95349121, 13.18139839, 15.18139839,
        8.        , 68.18139648, 67.75968933, 67.54418182, 44.96899414,
       21.39379883,  9.19689941, 16.77829742, 23.39379883, 24.3782959 ,
       15.18449783, 16.77829742, 23.18139648, 19.77209663, 20.19689941,
       55.34729385, 27.77209473, 11.19999886, 43.57209015, 19.        ,
       13.99999809, 19.19689941, 26.1689949 , 39.20000076, 18.17829704,
       12.59999847, 23.57519531, 21.79689598, 24.16589546, 42.17209244,
        5.5999999 , 39.36279297, 45.33798981, 23.36899567, 53.        ,
       45.59069824, 23.58449554, 54.93798828, 63.13489151, 13.98449898,
       45.35969162, 15.39379978, 37.1968956 , 15.18449783, 24.3782959 ,
       21.77519608, 21.57519722, 11.59070015, 21.17209625, 4.39379978,  8.39379978, 55.57519531,  5.        , 10.78759956,
       14.78759956,  3.59689999, 45.16589355, 17.98449707, 10.        ,
       90.15971375,  4.9968996 , 34.3782959 , 30.16589355, 23.18139648,
       41.54418945, 30.19689369,  1.        , 21.19379616,  9.19689941,
       11.19999886,  5.79379988, 10.98449898, 20.77209663, 51.77829361,
       18.39379692, 24.57519531, 40.1906929 ,  1.        , 20.78759956,
       33.57829285,  2.79999995, 12.3844986 , 30.16589355, 16.39379883,
       48.54418945, 39.96899414, 28.36589432, 40.95349121, 34.37519455,
       23.39379883, 35.95349121, 33.19689941, 13.39379883, 15.59379768,
       20.78759766,  3.59689999, 43.16589355,  1.39999998, 21.57519722,
       10.18759918, 19.19069672, 17.38139725, 33.19689941,  8.78759956,
        7.39379978,  7.7968998 , 15.37829781, 12.        , 17.19689941,
       61.74108887, 16.18139839, 19.78759766, 24.57519531, 40.36279297,
       37.15349197, 32.19689941, 24.3782959 , 15.18139839, 43.18139648,
       26.57519531, 26.9875946 , 63.34729004, 18.18139839, 23.18139648,
       42.3782959 ,  6.19689989, 51.13179016,  8.78759956, 54.15039062,
       26.79379463, 45.33798981, 29.19689941, 21.59069824, 19.18139648,
        9.39069939, 30.7565937 , 16.18139839, 15.78759766,  2.19689989,
       32.19999313, 17.7906971 , 25.58759499, 12.3844986 ,  7.7968998 ,
       27.98449707, 33.98449707, 26.16589546, 20.77209663, 35.17209244,
       18.39379692, 37.97209167, 35.77209473, 16.19689751, 19.79379654,
       27.19689941,  9.39379978, 18.59069824, 27.77209473, 25.58759499,
       14.19689941, 15.98139763, 31.3782959 , 17.7906971 , 37.3782959 ,
        6.19689989,  9.39379978, 11.19689941, 19.57519722,  9.39069939,
       37.34729385, 38.55969238, 37.96899414, 9.78759956, 38.18759155, 29.3782959 , 28.36589432, 13.59069824,
       10.59070015, 24.18139648,  3.19689989, 55.74108887, 29.98449707,
       15.39999771, 10.79069901, 11.39379883, 11.98449898,  3.        ,
       15.59070015, 23.77209663, 50.34729004, 22.57519722, 36.57209396,
       35.59069824, 11.98449898, 28.77209473, 26.57829475, 21.3782959 ,
       10.19689941,  1.39999998, 21.19689941, 45.7565918 , 20.77209663,
       30.98449707, 16.98449707, 31.16589355,  5.19689989, 10.18759918,
        2.79999995, 13.        ,  5.39379978, 32.78759766, 34.16589355,
       12.3844986 , 29.37829399, 16.19689751, 12.78759956, 15.98139763,
        5.79379988,  7.99069977,  4.9968996 , 24.16589546,  5.19689989,
       57.55969238, 29.55969429, 21.19379616, 15.        , 47.77209473,
       54.98449707,  2.79999995, 57.19689941, 62.78759766, 48.39379883,
        7.        , 25.18139839,  9.39379978, 33.98449707, 63.57519531,
        4.19689989, 13.99999809, 19.18139648, 53.36279297, 40.34729385,
       48.36279297, 37.16589355,  9.59070015,  1.39999998, 12.79379845,
       14.38759804, 16.78759766,  9.39069939, 13.39379883,  5.19689989,
        3.19689989, 11.98449898, 15.39379978, 23.39379883, 21.98449707,
        2.19689989, 10.39379883,  6.3968997 ,  9.19689941, 38.74729156,
       13.        , 21.77519608, 34.16589355, 13.39379883, 67.36279297,
        1.        , 15.59379768, 76.51318359, 43.37829208, 23.77209663,
       39.3782959 , 20.37829781,  4.19689989, 10.79069901, 27.19689941,
       16.58449745, 19.78759766, 26.16589546, 46.78139114, 15.        ,
       48.13798904, 60.98449707, 33.57519531, 51.59069824, 52.98449707,
       30.39379883,  5.79379988, 13.        , 23.17519569, 18.78759766,
       13.19689941, 20.98449707, 14.19689941,  3.59689999, 17.18139839,
       20.97829628, 15.39379883, 11.        , 10.19689941, 12.19689941,
       13.18139839,  8.19689941, 89.70697784, 16.58449745,  38.74729156,  59.18761063,  82.08837891,
       113.04187775,  66.12248993, 100.67907715, 118.26667023,
       115.59697723, 135.75662231,  63.18139648, 106.05427551,
       123.24187469,  41.3782959 ,  97.92248535,  17.18139839,
        84.96899414,  66.18139648,  41.78759766, 170.55969238,
        24.79069519,  29.59069824, 202.86047363, 114.86048126,
       198.8449707 ,   6.59069967, 151.78759766,  12.98759842,
       135.71318054,  51.1348877 ,  92.5721283 ,  64.55969238,
        76.33177948,  94.57523346,  70.77209473,  68.16589355,
        65.16589355,  55.59070969,  56.17209244,  37.96899414,
        29.16589546,   4.39379978,  45.18139648,  14.        ,
        25.16589546,  27.97829437,  26.96899605,  39.75969315,
        55.74108887,  47.39379883,  50.55969238,  51.77209473,
        42.3782959 ,  37.77829361,  57.74108887,  81.34729004,
        51.18139648,   9.78759956, 103.7565918 , 102.08837891,
       128.78759766,  38.78759766, 131.48217773,  56.96899033,
        80.08837891,  61.39379883,  70.77209473,  62.71938705,
        78.54418945,  63.17210007,  43.76589203,  19.37829781,
       101.1659317 ,  41.18139648,  56.58140182,  88.28527832,
        83.08837891,  35.98449707,  16.        ,  29.77209473,
        52.55969238,  25.        ,  18.        , 9.19689941, 17.19689941, 23.79999542, 19.79379654, 18.57519722,
       24.16589546, 14.98449898, 18.59069824, 21.77519608, 35.16589355,
       24.18139648, 25.78759766, 23.79999542, 24.96899605, 46.17829132,
        3.59689999, 31.96279335, 51.54418945, 19.98759651, 12.59069824,
       50.19689941, 73.74108887, 73.36279297, 73.59069824, 15.78759766,
       15.78759766, 41.3782959 , 51.39070511, 25.58759499, 34.59069443,
       30.39379883, 40.55969238, 16.78759766, 11.99689865, 35.59069824,
       20.98449707, 37.36279297, 14.18139839, 19.77209663, 33.9875946 ,
       33.18139648, 11.78759956, 85.14109039, 70.51318359, 47.7565918 ,
       57.72558594, 50.35659027, 43.77209473, 54.34108734, 93.7565918 ,
       50.13489151, 18.97519684, 37.58449173, 40.54419327, 13.78759956,
       54.36279297, 18.37829781, 19.39379883, 26.        , 29.19689941,
       21.5813961 , 20.37519646,  7.99069977, 16.78759766, 13.18139839,
       29.59379387, 16.77829742,  7.59069967, 17.59689713,  7.59069967,
       20.37519646,  3.59689999, 16.39379883,  9.        , 18.98449707,
       26.3782959 , 11.98449898, 25.37209511, 40.98759079,  2.19689989,
       16.19689941, 15.37829781,  4.19689989, 36.19689941, 16.19689751,
       29.18139648, 16.39379883,  2.19689989, 14.19379807, 62.34729004,
       32.96899414, 38.19689941, 16.98449707, 69.10388947, 32.77209473,
       53.3782959 , 93.08837891, 28.77519417, 24.16589546, 18.39379692,
       16.19689941, 14.18139839, 38.55969238, 27.39379883,  2.19689989])
kstest(x_off, y_off)

KstestResult(statistic=0.06421152030217187, pvalue=0.025370239168282294)

In [164]:
np.size(x_off)

1059

In [165]:
np.size(y_off)

1059